In [ ]:
import numpy as np
import scipy
import sklearn.manifold
import sklearn.decomposition
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import umap
import pandas
import pickle
import os
import collections

In [ ]:
INPUT = '../../histnorm/datasets/historical/german/german-anselm.test.txt'
ENCODING = 'utf-8'
CORPUS_NAME = 'german-anselm'

# Loading input file, which has the original and modernised token in each line separated by a \t
with open(INPUT, 'r', encoding=ENCODING) as infile:
    tokens = [line.strip().split('\t') for line in infile]

In [ ]:
# Getting the original and modernised tokens and types
tokens_original = [token[0] for token in tokens[:5000]]
tokens_modernised = [token[1] for token in tokens[:5000]]
types_original = list(set(tokens_original))
types_modernised = list(set(tokens_modernised))

In [ ]:
# Levenshtein Distance
def levenshtein(string1, string2):
    if string1 == string2:
        return 0

    if not string2:
        return len(string1)
    if not string1:
        return len(string2)

    rows = len(string1) + 1
    cols = len(string2) + 1
    dist = [[0 for c in range(cols)] for r in range(rows)]

    for j in range(1, rows):
        dist[j][0] = j
    for i in range(1, cols):
        dist[0][i] = i

    for col in range(1, cols):
        for row in range(1, rows):
            cost = 1
            if string1[row - 1] == string2[col - 1]:
                cost = 0
            dist[row][col] = min(dist[row - 1][col] + 1, dist[row][col - 1] + 1, dist[row - 1][col - 1] + cost)

    # Enable for Debugging
    # print('\n'.join([''.join(['{:4}'.format(elem) for elem in row]) for row in dist]))
    return dist[row][col]

assert levenshtein('', '') == 0
assert levenshtein('foobar', 'foobar') == 0
assert levenshtein('foobar', 'foubar') == 1
assert levenshtein('foobar', 'fuubar') == 2
assert levenshtein('foobar', 'fuuar') == 3
assert levenshtein('foobar', '') == 6

In [ ]:
# Jaro Similarily
def jaro(string1, string2):

    length1 = len(string1)
    length2 = len(string2)
   
    if length1 == 0:
        return 0.0
    
    if string1 == string2:
        return 1.0   

    match_bound = max(length1, length2) // 2 - 1

    matches = 0  
    transpositions = 0

    flagged_1 = [] 
    flagged_2 = []

    for i in range(length1):
        upperbound = min(i + match_bound, length2 - 1)
        lowerbound = max(0, i - match_bound)
        for j in range(lowerbound, upperbound + 1):
            if string1[i] == string2[j] and j not in flagged_2:
                matches += 1
                flagged_1.append(i)
                flagged_2.append(j)
                break

    flagged_2.sort()

    for i, j in zip(flagged_1, flagged_2):
        if string1[i] != string2[j]:
            transpositions += 1

    if matches == 0:
        return 0.0

    return (1/3 * ( matches / length1 + matches / length2 + (matches - transpositions // 2) / matches))

assert jaro('', '') == 0.0
assert jaro('foobar', '') == 0.0
assert jaro('foobar', 'foobar') == 1.0
assert jaro('foobar', 'barfoo') == 0.4444444444444444
assert jaro('duane', 'dwayne') == 0.8222222222222222
assert jaro('hans', 'gruber') == 0.0

In [ ]:
# IBM (LCS-Levenshtein Normalized)

# Contractor, D., Faruquie, T. A., & Subramaniam, L. V. (2010, August). 
# Unsupervised cleansing of noisy text. 
# In Proceedings of the 23rd International Conference on Computational Linguistics:
# Posters (pp. 189-196). Association for Computational Linguistics.

from itertools import groupby

# Longest Common Substring
def longest_common_string(string1, string2):
    if string1 == string2:
        return len(string1)

    if not string1 or not string2:
        return 0
    
    rows = len(string1) + 1
    cols = len(string2) + 1
    table = [[0 for c in range(cols)] for r in range(rows)]

    longest = 0
    for col in range(cols):
        for row in range(rows):
            if col == 0 and row == 0:
                table[row][col] = 0
            if string1[row - 1] == string2[col - 1]:
                table[row][col] = table[row - 1][col - 1] + 1
                longest = max(longest, table[row][col])
            else:
                table[row][col] = 0
    
    return longest

assert longest_common_string('', '') == 0
assert longest_common_string('foobar', '') == 0
assert longest_common_string('foobar', 'foobar') == 6
assert longest_common_string('foobar', 'foo') == 3
assert longest_common_string('foobar', 'f') == 1


def lcs_ratio(string1, string2):
    if not string1 or not string2:
        return 0.0
    ratio = longest_common_string(string1, string2) / len(string1)
    return ratio

assert lcs_ratio('', '') == 0.0
assert lcs_ratio('foo', '') == 0.0
assert lcs_ratio('foobar', 'foobar') == 1.0
assert lcs_ratio('foo', 'bar') == 0.0
assert lcs_ratio('word', 'deoxyribonucleic') == 0.25


def consonant_skeleton(string, vowels='aeiouy'):
    without_vowels = ''.join([char for char in string if char not in vowels])     
    deduplicated_consonants = ''.join(char for char, _ in groupby(without_vowels))
    return deduplicated_consonants

assert consonant_skeleton('') == ''
assert consonant_skeleton('aeio') == ''
assert consonant_skeleton('foobar') == 'fbr'
assert consonant_skeleton('ffoobbar') == 'fbr'
assert consonant_skeleton('barfoobar') == 'brfbr'


def ibm_similarity(string1, string2):
    similarity = lcs_ratio(string1, string2) / (levenshtein (consonant_skeleton(string1), consonant_skeleton(string2)) + 1)
    return similarity

assert ibm_similarity('', '') == 0.0
assert ibm_similarity('foobar', '') == 0.0
assert ibm_similarity('foobar', 'foobar') == 1.0
assert ibm_similarity('foo', 'bar') == 0.0
assert ibm_similarity('word', 'deoxyribonucleic') == 0.03125
assert ibm_similarity('foobar', 'aeiou') == 0.041666666666666664

In [ ]:
# Getting the original and modernised tokens and types
tmp_tokens_original = [token[0] for token in tokens if token[1].startswith('mein')]
tmp_tokens_modernised = [token[1] for token in tokens if token[1].startswith('mein')]
tmp_types_original = list(set(tmp_tokens_original))
tmp_types_modernised = list(set(tmp_tokens_modernised))

tmp_types_original_reshaped = np.array(tmp_types_original).reshape(-1,1)
tmp_types_original_pairwise_distance_levenshtein = scipy.spatial.distance.pdist(tmp_types_original_reshaped, lambda x,y: levenshtein(str(x[0]),str(y[0])))
tmp_types_original_pairwise_distance_jaro = scipy.spatial.distance.pdist(tmp_types_original_reshaped, lambda x,y: jaro(str(x[0]),str(y[0])))
tmp_types_original_pairwise_distance_ibm = scipy.spatial.distance.pdist(tmp_types_original_reshaped, lambda x,y: ibm_similarity(str(x[0]),str(y[0])))


tmp_original_distance_matrix_levenshtein = pandas.DataFrame(scipy.spatial.distance.squareform(tmp_types_original_pairwise_distance_levenshtein), index=tmp_types_original, columns=tmp_types_original)
tmp_original_distance_matrix_jaro = pandas.DataFrame(scipy.spatial.distance.squareform(tmp_types_original_pairwise_distance_jaro), index=tmp_types_original, columns=tmp_types_original)
tmp_original_distance_matrix_ibm = pandas.DataFrame(scipy.spatial.distance.squareform(tmp_types_original_pairwise_distance_ibm), index=tmp_types_original, columns=tmp_types_original)


tmp_types_modernised_reshaped = np.array(tmp_types_modernised).reshape(-1,1)
tmp_types_modernised_pairwise_distance_levenshtein = scipy.spatial.distance.pdist(tmp_types_modernised_reshaped, lambda x,y: levenshtein(str(x[0]),str(y[0])))
tmp_types_modernised_pairwise_distance_jaro = scipy.spatial.distance.pdist(tmp_types_modernised_reshaped, lambda x,y: jaro(str(x[0]),str(y[0])))
tmp_types_modernised_pairwise_distance_ibm = scipy.spatial.distance.pdist(tmp_types_modernised_reshaped, lambda x,y: ibm_similarity(str(x[0]),str(y[0])))

tmp_modernised_distance_matrix_levenshtein = pandas.DataFrame(scipy.spatial.distance.squareform(tmp_types_modernised_pairwise_distance_levenshtein), index=tmp_types_modernised, columns=tmp_types_modernised)
tmp_modernised_distance_matrix_jaro = pandas.DataFrame(scipy.spatial.distance.squareform(tmp_types_modernised_pairwise_distance_jaro), index=tmp_types_modernised, columns=tmp_types_modernised)
tmp_modernised_distance_matrix_ibm = pandas.DataFrame(scipy.spatial.distance.squareform(tmp_types_modernised_pairwise_distance_ibm), index=tmp_types_modernised, columns=tmp_types_modernised)

In [ ]:
tmp_original_distance_matrix_levenshtein

In [ ]:
tmp_original_distance_matrix_jaro

In [ ]:
tmp_original_distance_matrix_ibm

In [ ]:
%%time

types_original_reshaped = np.array(types_original).reshape(-1,1)
types_original_pairwise_distance_levenshtein = scipy.spatial.distance.pdist(types_original_reshaped, lambda x,y: levenshtein(str(x[0]),str(y[0])))   
types_original_pairwise_distance_jaro = scipy.spatial.distance.pdist(types_original_reshaped, lambda x,y: jaro(str(x[0]),str(y[0])))   
types_original_pairwise_distance_ibm = scipy.spatial.distance.pdist(types_original_reshaped, lambda x,y: ibm_similarity(str(x[0]),str(y[0])))


types_modernised_reshaped = np.array(types_modernised).reshape(-1,1)
types_modernised_pairwise_distance_levenshtein = scipy.spatial.distance.pdist(types_modernised_reshaped, lambda x,y: levenshtein(str(x[0]),str(y[0]))) 
types_modernised_pairwise_distance_jaro = scipy.spatial.distance.pdist(types_modernised_reshaped, lambda x,y: jaro(str(x[0]),str(y[0])))
types_modernised_pairwise_distance_ibm = scipy.spatial.distance.pdist(types_modernised_reshaped, lambda x,y: ibm_similarity(str(x[0]),str(y[0])))   

In [ ]:
%%time
# Transforming pairwise distances into a full similarity matrix
original_distance_matrix_levenshtein = pandas.DataFrame(scipy.spatial.distance.squareform(types_original_pairwise_distance_levenshtein), index=types_original, columns=types_original)
modernised_distance_matrix_levenshtein = pandas.DataFrame(scipy.spatial.distance.squareform(types_modernised_pairwise_distance_levenshtein), index=types_modernised, columns=types_modernised)

original_distance_matrix_jaro = pandas.DataFrame(scipy.spatial.distance.squareform(types_original_pairwise_distance_jaro), index=types_original, columns=types_original)
modernised_distance_matrix_jaro = pandas.DataFrame(scipy.spatial.distance.squareform(types_modernised_pairwise_distance_jaro), index=types_modernised, columns=types_modernised)

original_distance_matrix_ibm = pandas.DataFrame(scipy.spatial.distance.squareform(types_original_pairwise_distance_ibm), index=types_original, columns=types_original)
modernised_distance_matrix_ibm = pandas.DataFrame(scipy.spatial.distance.squareform(types_modernised_pairwise_distance_ibm), index=types_modernised, columns=types_modernised)

# PCA 

In [ ]:
pca = sklearn.decomposition.PCA(n_components = 3)

original_pca_lev = pca.fit(original_distance_matrix_levenshtein).transform(original_distance_matrix_levenshtein)
modernised_pca_lev = pca.fit(modernised_distance_matrix_levenshtein).transform(modernised_distance_matrix_levenshtein)

original_pca_jaro = pca.fit(original_distance_matrix_jaro).transform(original_distance_matrix_jaro)
modernised_pca_jaro = pca.fit(modernised_distance_matrix_jaro).transform(modernised_distance_matrix_jaro)

original_pca_ibm = pca.fit(original_distance_matrix_ibm).transform(original_distance_matrix_ibm)
modernised_pca_ibm = pca.fit(modernised_distance_matrix_ibm).transform(modernised_distance_matrix_ibm)

In [ ]:
original_pca_levenshtein_df = pandas.DataFrame()
original_pca_levenshtein_df['pca-x-original'] = original_pca_lev[:,0]
original_pca_levenshtein_df['pca-y-original'] = original_pca_lev[:,1]
original_pca_levenshtein_df['pca-z-original'] = original_pca_lev[:,2]
original_pca_levenshtein_df['token'] = original_distance_matrix_levenshtein.index

modernised_pca_levenshtein_df = pandas.DataFrame()
modernised_pca_levenshtein_df['pca-x-modernised'] = modernised_pca_lev[:,0]
modernised_pca_levenshtein_df['pca-y-modernised'] = modernised_pca_lev[:,1]
modernised_pca_levenshtein_df['pca-z-modernised'] = modernised_pca_lev[:,2]
modernised_pca_levenshtein_df['token'] = modernised_distance_matrix_levenshtein.index

original_pca_jaro_df = pandas.DataFrame()
original_pca_jaro_df['pca-x-original'] = original_pca_jaro[:,0]
original_pca_jaro_df['pca-y-original'] = original_pca_jaro[:,1]
original_pca_jaro_df['pca-z-original'] = original_pca_jaro[:,2]
original_pca_jaro_df['token'] = original_distance_matrix_jaro.index

modernised_pca_jaro_df = pandas.DataFrame()
modernised_pca_jaro_df['pca-x-modernised'] = modernised_pca_jaro[:,0]
modernised_pca_jaro_df['pca-y-modernised'] = modernised_pca_jaro[:,1]
modernised_pca_jaro_df['pca-z-modernised'] = modernised_pca_jaro[:,2]
modernised_pca_jaro_df['token'] = modernised_distance_matrix_jaro.index

original_pca_ibm_df = pandas.DataFrame()
original_pca_ibm_df['pca-x-original'] = original_pca_ibm[:,0]
original_pca_ibm_df['pca-y-original'] = original_pca_ibm[:,1]
original_pca_ibm_df['pca-z-original'] = original_pca_ibm[:,2]
original_pca_ibm_df['token'] = original_distance_matrix_ibm.index

modernised_pca_ibm_df = pandas.DataFrame()
modernised_pca_ibm_df['pca-x-modernised'] = modernised_pca_ibm[:,0]
modernised_pca_ibm_df['pca-y-modernised'] = modernised_pca_ibm[:,1]
modernised_pca_ibm_df['pca-z-modernised'] = modernised_pca_ibm[:,2]
modernised_pca_ibm_df['token'] = modernised_distance_matrix_ibm.index

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=original_pca_levenshtein_df['pca-x-original'],
    y=original_pca_levenshtein_df['pca-y-original'],
    z=original_pca_levenshtein_df['pca-z-original'],
    mode='markers',
    text=original_pca_levenshtein_df['token'],
    marker=dict(
        size=2,
        color='steelblue',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=modernised_pca_levenshtein_df['pca-x-modernised'],
    y=modernised_pca_levenshtein_df['pca-y-modernised'],
    z=modernised_pca_levenshtein_df['pca-z-modernised'],
    mode='markers',
    text=modernised_pca_levenshtein_df['token'],
    marker=dict(
        size=2,
        color='skyblue',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=original_pca_jaro_df['pca-x-original'],
    y=original_pca_jaro_df['pca-y-original'],
    z=original_pca_jaro_df['pca-z-original'],
    mode='markers',
    text=original_pca_jaro_df['token'],
    marker=dict(
        size=2,
        color='salmon',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=original_pca_ibm_df['pca-x-original'],
    y=original_pca_ibm_df['pca-y-original'],
    z=original_pca_ibm_df['pca-z-original'],
    mode='markers',
    text=original_pca_ibm_df['token'],
    marker=dict(
        size=2,
        color='seagreen',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=modernised_pca_ibm_df['pca-x-modernised'],
    y=modernised_pca_ibm_df['pca-y-modernised'],
    z=modernised_pca_ibm_df['pca-z-modernised'],
    mode='markers',
    text=modernised_pca_ibm_df['token'],
    marker=dict(
        size=2,
        color='darkgreen',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

# UMAP

In [ ]:
%%time

original_similarity_embedded_umap_levenshtein = umap.UMAP(n_components=3).fit_transform(original_distance_matrix_levenshtein)
modernised_similarity_embedded_umap_levenshtein = umap.UMAP(n_components=3).fit_transform(modernised_distance_matrix_levenshtein)

original_similarity_embedded_umap_jaro = umap.UMAP(n_components=3).fit_transform(original_distance_matrix_jaro)
modernised_similarity_embedded_umap_jaro = umap.UMAP(n_components=3).fit_transform(modernised_distance_matrix_jaro)

original_similarity_embedded_umap_ibm = umap.UMAP(n_components=3).fit_transform(original_distance_matrix_ibm)
modernised_similarity_embedded_umap_ibm = umap.UMAP(n_components=3).fit_transform(modernised_distance_matrix_ibm)

In [ ]:
original_umap_levenshtein = pandas.DataFrame()
original_umap_levenshtein['umap-x-original'] = original_similarity_embedded_umap_levenshtein[:,0]
original_umap_levenshtein['umap-y-original'] = original_similarity_embedded_umap_levenshtein[:,1]
original_umap_levenshtein['umap-z-original'] = original_similarity_embedded_umap_levenshtein[:,2]
original_umap_levenshtein['token'] = original_distance_matrix_levenshtein.index

modernised_umap_levenshtein = pandas.DataFrame()
modernised_umap_levenshtein['umap-x-modernised'] = modernised_similarity_embedded_umap_levenshtein[:,0]
modernised_umap_levenshtein['umap-y-modernised'] = modernised_similarity_embedded_umap_levenshtein[:,1]
modernised_umap_levenshtein['umap-z-modernised'] = modernised_similarity_embedded_umap_levenshtein[:,2]
modernised_umap_levenshtein['token'] = modernised_distance_matrix_levenshtein.index

original_umap_jaro = pandas.DataFrame()
original_umap_jaro['umap-x-original'] = original_similarity_embedded_umap_jaro[:,0]
original_umap_jaro['umap-y-original'] = original_similarity_embedded_umap_jaro[:,1]
original_umap_jaro['umap-z-original'] = original_similarity_embedded_umap_jaro[:,2]
original_umap_jaro['token'] = original_distance_matrix_jaro.index

modernised_umap_jaro = pandas.DataFrame()
modernised_umap_jaro['umap-x-modernised'] = modernised_similarity_embedded_umap_jaro[:,0]
modernised_umap_jaro['umap-y-modernised'] = modernised_similarity_embedded_umap_jaro[:,1]
modernised_umap_jaro['umap-z-modernised'] = modernised_similarity_embedded_umap_jaro[:,2]
modernised_umap_jaro['token'] = modernised_distance_matrix_jaro.index

original_umap_ibm = pandas.DataFrame()
original_umap_ibm['umap-x-original'] = original_similarity_embedded_umap_ibm[:,0]
original_umap_ibm['umap-y-original'] = original_similarity_embedded_umap_ibm[:,1]
original_umap_ibm['umap-z-original'] = original_similarity_embedded_umap_ibm[:,2]
original_umap_ibm['token'] = original_distance_matrix_ibm.index

modernised_umap_ibm = pandas.DataFrame()
modernised_umap_ibm['umap-x-modernised'] = modernised_similarity_embedded_umap_ibm[:,0]
modernised_umap_ibm['umap-y-modernised'] = modernised_similarity_embedded_umap_ibm[:,1]
modernised_umap_ibm['umap-z-modernised'] = modernised_similarity_embedded_umap_ibm[:,2]
modernised_umap_ibm['token'] = modernised_distance_matrix_ibm.index

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=original_umap_levenshtein['umap-x-original'],
    y=original_umap_levenshtein['umap-y-original'],
    z=original_umap_levenshtein['umap-z-original'],
    mode='markers',
    text=original_umap_levenshtein['token'],
    marker=dict(
        size=2,
        color='steelblue',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=modernised_umap_levenshtein['umap-x-modernised'],
    y=modernised_umap_levenshtein['umap-y-modernised'],
    z=modernised_umap_levenshtein['umap-z-modernised'],
    mode='markers',
    text=modernised_umap_levenshtein['token'],
    marker=dict(
        size=2,
        color='skyblue',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=original_umap_jaro['umap-x-original'],
    y=original_umap_jaro['umap-y-original'],
    z=original_umap_jaro['umap-z-original'],
    mode='markers',
    text=original_umap_jaro['token'],
    marker=dict(
        size=2,
        color='salmon',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=modernised_umap_jaro['umap-x-modernised'],
    y=modernised_umap_jaro['umap-y-modernised'],
    z=modernised_umap_jaro['umap-z-modernised'],
    mode='markers',
    text=modernised_umap_jaro['token'],
    marker=dict(
        size=2,
        color='orange',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=original_umap_ibm['umap-x-original'],
    y=original_umap_ibm['umap-y-original'],
    z=original_umap_ibm['umap-z-original'],
    mode='markers',
    text=original_umap_ibm['token'],
    marker=dict(
        size=2,
        color='seagreen',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=modernised_umap_ibm['umap-x-modernised'],
    y=modernised_umap_ibm['umap-y-modernised'],
    z=modernised_umap_ibm['umap-z-modernised'],
    mode='markers',
    text=modernised_umap_ibm['token'],
    marker=dict(
        size=2,
        color='darkgreen',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

# t-SNE

In [ ]:
%%time
original_similarity_embedded_tsne_levenshtein = sklearn.manifold.TSNE(n_components=3).fit_transform(original_distance_matrix_levenshtein)
modernised_similarity_embedded_tsne_levenshtein = sklearn.manifold.TSNE(n_components=3).fit_transform(modernised_distance_matrix_levenshtein)

original_similarity_embedded_tsne_jaro = sklearn.manifold.TSNE(n_components=3).fit_transform(original_distance_matrix_jaro)
modernised_similarity_embedded_tsne_jaro = sklearn.manifold.TSNE(n_components=3).fit_transform(modernised_distance_matrix_jaro)

original_similarity_embedded_tsne_ibm = sklearn.manifold.TSNE(n_components=3).fit_transform(original_distance_matrix_ibm)
modernised_similarity_embedded_tsne_ibm = sklearn.manifold.TSNE(n_components=3).fit_transform(modernised_distance_matrix_ibm)

In [ ]:
original_tsne_levenshtein = pandas.DataFrame()
original_tsne_levenshtein['tsne-x-original'] = original_similarity_embedded_tsne_levenshtein[:,0]
original_tsne_levenshtein['tsne-y-original'] = original_similarity_embedded_tsne_levenshtein[:,1]
original_tsne_levenshtein['tsne-z-original'] = original_similarity_embedded_tsne_levenshtein[:,2]
original_tsne_levenshtein['token'] = original_distance_matrix_levenshtein.index

modernised_tsne_levenshtein = pandas.DataFrame()
modernised_tsne_levenshtein['tsne-x-modernised'] = modernised_similarity_embedded_tsne_levenshtein[:,0]
modernised_tsne_levenshtein['tsne-y-modernised'] = modernised_similarity_embedded_tsne_levenshtein[:,1]
modernised_tsne_levenshtein['tsne-z-modernised'] = modernised_similarity_embedded_tsne_levenshtein[:,2]
modernised_tsne_levenshtein['token'] = modernised_distance_matrix_levenshtein.index

original_tsne_jaro = pandas.DataFrame()
original_tsne_jaro['tsne-x-original'] = original_similarity_embedded_tsne_jaro[:,0]
original_tsne_jaro['tsne-y-original'] = original_similarity_embedded_tsne_jaro[:,1]
original_tsne_jaro['tsne-z-original'] = original_similarity_embedded_tsne_jaro[:,2]
original_tsne_jaro['token'] = original_distance_matrix_jaro.index

modernised_tsne_jaro = pandas.DataFrame()
modernised_tsne_jaro['tsne-x-modernised'] = modernised_similarity_embedded_tsne_jaro[:,0]
modernised_tsne_jaro['tsne-y-modernised'] = modernised_similarity_embedded_tsne_jaro[:,1]
modernised_tsne_jaro['tsne-z-modernised'] = modernised_similarity_embedded_tsne_jaro[:,2]
modernised_tsne_jaro['token'] = modernised_distance_matrix_jaro.index

original_tsne_ibm = pandas.DataFrame()
original_tsne_ibm['tsne-x-original'] = original_similarity_embedded_tsne_ibm[:,0]
original_tsne_ibm['tsne-y-original'] = original_similarity_embedded_tsne_ibm[:,1]
original_tsne_ibm['tsne-z-original'] = original_similarity_embedded_tsne_ibm[:,2]
original_tsne_ibm['token'] = original_distance_matrix_ibm.index

modernised_tsne_ibm = pandas.DataFrame()
modernised_tsne_ibm['tsne-x-modernised'] = modernised_similarity_embedded_tsne_ibm[:,0]
modernised_tsne_ibm['tsne-y-modernised'] = modernised_similarity_embedded_tsne_ibm[:,1]
modernised_tsne_ibm['tsne-z-modernised'] = modernised_similarity_embedded_tsne_ibm[:,2]
modernised_tsne_ibm['token'] = modernised_distance_matrix_ibm.index

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=original_tsne_levenshtein['tsne-x-original'],
    y=original_tsne_levenshtein['tsne-y-original'],
    z=original_tsne_levenshtein['tsne-z-original'],
    mode='markers',
    text=original_tsne_levenshtein['token'],
    marker=dict(
        size=2,
        color='steelblue',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=modernised_tsne_levenshtein['tsne-x-modernised'],
    y=modernised_tsne_levenshtein['tsne-y-modernised'],
    z=modernised_tsne_levenshtein['tsne-z-modernised'],
    mode='markers',
    text=modernised_umap_levenshtein['token'],
    marker=dict(
        size=2,
        color='skyblue',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=original_tsne_jaro['tsne-x-original'],
    y=original_tsne_jaro['tsne-y-original'],
    z=original_tsne_jaro['tsne-z-original'],
    mode='markers',
    text=original_tsne_jaro['token'],
    marker=dict(
        size=2,
        color='salmon',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=modernised_tsne_jaro['tsne-x-modernised'],
    y=modernised_tsne_jaro['tsne-y-modernised'],
    z=modernised_tsne_jaro['tsne-z-modernised'],
    mode='markers',
    text=modernised_tsne_jaro['token'],
    marker=dict(
        size=2,
        color='orange',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=original_tsne_ibm['tsne-x-original'],
    y=original_tsne_ibm['tsne-y-original'],
    z=original_tsne_ibm['tsne-z-original'],
    mode='markers',
    text=original_tsne_ibm['token'],
    marker=dict(
        size=2,
        color='seagreen',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=modernised_tsne_ibm['tsne-x-modernised'],
    y=modernised_tsne_ibm['tsne-y-modernised'],
    z=modernised_tsne_ibm['tsne-z-modernised'],
    mode='markers',
    text=modernised_tsne_ibm['token'],
    marker=dict(
        size=2,
        color='darkgreen',
        opacity=1
    )
)])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()